In [1]:
import os, shutil

In [2]:
import tensorflow as tf

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
from keras import layers
from keras import models
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
import keras

Using TensorFlow backend.


In [4]:
import numpy as np

In [5]:
from PIL import Image

In [6]:
from sklearn.metrics import classification_report, confusion_matrix

In [7]:
from keras.preprocessing import image

In [8]:
import time

# Create directories

In [10]:
original_dataset_dir = 'C:/Users/bened/OneDrive/Arbeit/Lernen/python_training/train_pics'

In [11]:
base_dir = 'C:/Users/bened/OneDrive/Arbeit/Lernen/python_training/data_small/'
#os.mkdir(base_dir)

In [12]:
train_dir = os.path.join(base_dir, 'train')
#os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
#os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
#os.mkdir(test_dir)

In [13]:
train_jump_dir = os.path.join(train_dir, 'jump')
#os.mkdir(train_jump_dir)
train_nojump_dir = os.path.join(train_dir, 'nojump')
#os.mkdir(train_nojump_dir)

In [14]:
validation_jump_dir = os.path.join(validation_dir, 'jump')
#os.mkdir(validation_jump_dir)
validation_nojump_dir = os.path.join(validation_dir, 'nojump')
#os.mkdir(validation_nojump_dir)

In [15]:
test_jump_dir = os.path.join(test_dir, 'jump')
#os.mkdir(test_jump_dir)
test_nojump_dir = os.path.join(test_dir, 'nojump')
#os.mkdir(test_nojump_dir)

In [16]:
print("total training jump images:", len(os.listdir(train_jump_dir)))
print("total training nojump images:", len(os.listdir(train_nojump_dir)))
print("total validation jump images:", len(os.listdir(validation_jump_dir)))
print("total validation nojump images:", len(os.listdir(validation_nojump_dir)))
print("total test jump images:", len(os.listdir(test_jump_dir)))
print("total test nojump images:", len(os.listdir(test_nojump_dir)))

total training jump images: 267
total training nojump images: 269
total validation jump images: 55
total validation nojump images: 81
total test jump images: 24
total test nojump images: 197


In [26]:
model = models.Sequential()
model.add(layers.Conv2D(32, (10,10), activation = "relu", input_shape=(250, 75, 3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (10,10), activation = "relu"))
model.add(layers.MaxPooling2D((2,2)))
#model.add(layers.Conv2D(128, (10,10), activation="relu"))
#model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128, (5,5), activation="relu"))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation="relu"))
model.add(layers.Dense(1, activation = "sigmoid"))

In [27]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 241, 66, 32)       9632      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 120, 33, 32)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 111, 24, 64)       204864    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 55, 12, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 51, 8, 128)        204928    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 25, 4, 128)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 12800)             0         
__________

In [28]:
model.compile(loss="binary_crossentropy",
             optimizer = optimizers.RMSprop(lr=1e-4),
             metrics=["acc"])

In [52]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(250, 75),
        batch_size=20,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(250, 75),
        batch_size=10,
        class_mode='binary')

Found 658 images belonging to 2 classes.
Found 136 images belonging to 2 classes.


In [53]:
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

data batch shape: (20, 250, 75, 3)
labels batch shape: (20,)


In [54]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=3,
      validation_data=validation_generator,
      validation_steps=50)

Epoch 1/3
100/100 [==============================] - 200s 2s/step - loss: 0.1112 - acc: 0.9649 - val_loss: 0.1606 - val_acc: 0.9611
Epoch 2/3
100/100 [==============================] - 205s 2s/step - loss: 0.0943 - acc: 0.9678 - val_loss: 0.1538 - val_acc: 0.9711
Epoch 3/3
100/100 [==============================] - 491s 5s/step - loss: 0.0873 - acc: 0.9715 - val_loss: 0.1799 - val_acc: 0.9754


In [55]:
history.history

{'acc': [0.9648946766389409, 0.9679037049158645, 0.9714142377063288],
 'loss': [0.11124422603894918, 0.09441217847183392, 0.08749492967455978],
 'val_acc': [0.9610655657092079, 0.9710743757318859, 0.9754098316685098],
 'val_loss': [0.16058151964212722, 0.15380196224595996, 0.17989352958841767]}

In [56]:
os.chdir("C:/Users/bened/OneDrive/Arbeit/Lernen/python_training/")
model.save('jump_nojump_v2.h5')

In [18]:
os.chdir("C:/Users/bened/OneDrive/Arbeit/Lernen/python_training/")
model = load_model('jump_nojump_v1.h5')

In [57]:
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(250, 75),
        batch_size=221,
        class_mode='binary',
        shuffle = False)


Found 221 images belonging to 2 classes.


In [58]:
test_loss, test_acc = model.evaluate_generator(test_generator, steps=50)
print('test acc:', test_acc)

test acc: 0.9457013607025146


In [59]:
preds = model.predict_generator(test_generator, steps = 1)

In [26]:
preds_r = preds.round()
np.unique(preds_r, return_counts=True)

(array([0., 1.], dtype=float32), array([  1, 103], dtype=int64))

In [60]:
preds.round()

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],

In [46]:
test_generator.classes

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1])

In [29]:
file = "data_small/test/jump/direction_up171.png"

In [51]:
start = time.time()
file = "data_small/test/jump/direction_up338.png"
img = image.load_img(file, target_size = (250, 75))
img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor /= 255.

model.predict(img_tensor)[0,0]
end = time.time()
print(end - start)

0.04702138900756836


## Runtime testing